In [2]:
#@title Colab Mount
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3"
# !git clone "https://85869b109f25ac5241470005fcd8ead1673b1329@github.com/rohanrajpal/dlassignment1.git"
%cd "{PROJECT_PATH}"
!git config --global user.email "rohan17089@iiitd.ac.in"
!git config --global user.name "Rohan Rajpal"

%load_ext autoreload
# %cd ..

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3


In [0]:
#@title Password for SSHD
#1 - setup ssh/user 


#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

username: root
password:  JrBRhxSlTumD3wRyAb624xVB63L5Qv


In [0]:
#@title Setup Ngork
# Collapse
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')


Get your authtoken from https://dashboard.ngrok.com/auth
··········


In [0]:
#@title Git Stuff
# %cd q3
# !git pull origin master
# !git commit -m "ok"

# !git status
# !git add Q3.ipynb
# !git add Q3-Rohan.ipynb
# !git commit -m "ok"
# !git add q3/Q3.ipynb
# !git add q3/Old-Q3.ipynb
# !git commit -m "Add notebooks"
!git push origin master
# !git stash 
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3"

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/rohanrajpal/dlassignment1
 * branch            master     -> FETCH_HEAD
   00ac8de..7c352e4  master     -> origin/master
Updating 00ac8de..7c352e4
Fast-forward
 q3/visualize.py | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)


In [0]:
#@title Imports
import numpy as np
import torch
import torch.nn as nn

from retinanet import model
from retinanet.dataloader import CocoDataset,Normalizer,Resizer,Augmenter,AspectRatioBasedSampler,collater,CSVDataset, UnNormalizer
from torchvision import transforms
from retinanet import coco_eval,csv_eval
from importlib import reload
from torch.utils.data import DataLoader
import json,csv
from pprint import pprint
import time
import cv2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import torch.optim as optim
import argparse
import collections


In [0]:
PATH_TO_DATASET = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data"
PATH_TO_WEIGHTS = PATH_TO_DATASET + "/pretrained_weights_cleaned.pt"
ANNOTATION_CSV ="/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv"
CLASS_LIST ="/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/class_list.csv"

IMAGE_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images/train"
INSTANCES_TRAIN_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train.json"
INSTANCES_TRAIN_ALT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_alt.json"

TRAIN_CSV = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/train.csv"
VAL_CSV = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/val.csv"
PATH_TO_FINETUNED_WEIGHTS = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/model_final.pt"
BEST_WEIGHTS = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/third/csv_retinanet_best_model_mAP.pt"
TO_SAVE_WT = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/third/best_weigths.pt"

# !cat "{ANNOTATION_CSV}"

In [0]:
# Helper functions
def print_dict(model):
  print("Model's state_dict:")
  for param_tensor in model.state_dict():
      print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [0]:
#@title Clean given weights
pretrained_dict = torch.load(PATH_TO_WEIGHTS)
retina_dict = retinanet.state_dict()

def print_last(comment):
  print(comment,pretrained_dict["classificationModel.output.weight"].size())
  print(comment,pretrained_dict["classificationModel.output.bias"].size())

for k, v in pretrained_dict.items():
  if k in retina_dict:
    if k == "classificationModel.output.weight":
      #do something
      v = v[:72, :, :, :]
    elif k =="classificationModel.output.bias":
      v = v[:72]

    pretrained_dict[k] = v
# 2. overwrite entries in the existing state dict
retina_dict.update(pretrained_dict) 
# 3. load the new state dict
retinanet.load_state_dict(pretrained_dict)

# torch.save(retinanet.state_dict(), PATH_TO_DATASET + "/pretrained_weights_cleaned.pt")
# print_dict(retinanet)
# print(givenmodel['classificationModel.output.weight'])


In [0]:
#@title Part 1: mAP and Confusion Matrix
#@markdown The
%autoreload 2

dataset_val = CSVDataset(train_file = VAL_CSV, class_list=CLASS_LIST,
                                     transform=transforms.Compose([Normalizer(),Resizer()]))
retinanet = model.resnet50(num_classes=dataset_val.num_classes(),pretrained=False)

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))

retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = False
retinanet.eval()
retinanet.module.freeze_bn()

mAP = csv_eval.evaluate(dataset_val, retinanet)
print(mAP)

506/506
mAP:
bird: 0.0
bobcat: 0.011557311096481603
car: 0.4445338715265824
cat: 0.007590241576459477
raccoon: 0.05251340699101893
rabbit: 0.03546910755148741
coyote: 0.005090281855549031
squirrel: 0.0
{0: (0.0, 42.0), 1: (0.011557311096481603, 69.0), 2: (0.4445338715265824, 48.0), 3: (0.007590241576459477, 87.0), 4: (0.05251340699101893, 134.0), 5: (0.03546910755148741, 46.0), 6: (0.005090281855549031, 40.0), 7: (0.0, 56.0)}


In [0]:
#@title Layer Freezing
# retinanet = model.resnet50(num_classes=dataset_val.num_classes(),pretrained=False)
# retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))

# for layer in retinanet.modules():
#   if not isinstance(layer, model.ClassificationModel):
#       for param in layer.parameters():
#           param.requires_grad = False
#       print(layer)
#       print("-----------------")

# for layer in retinanet.modules():
#   if not isinstance(layer, model.ClassificationModel):
#       for param in layer.parameters():
#           print(param.requires_grad)
#       print(layer)
#       print("-----------------")
# for param in retinanet.parameters():
#   param.requires_grad = False
# for param in retinanet.classificationModel.parameters():
#   param.requires_grad = True
# cnt=0
# for name in retinanet.children():
#     # print(name)
#     cnt+=1
# print(cnt)

# num_ftrs = retinanet.fc.in_features


In [0]:
#@title Make Reduced JSON

# Make Reduced JSON

IMAGE_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images/train/"

# with open(INSTANCES_TRAIN_PATH) as f:
#   data = json.load(f)

# data_final = {}

# data_final["info"] = data["info"]
# data_final["categories"] = data["categories"]
# data_final["images"] = []
# data_final["annotations"] = []
# import os

# root = IMAGE_PATH
# file_map = {}
# for path, subdirs, files in os.walk(root):
#     if files:
#         print(os.path.join(path, min(files)))
#         for file in files:
#           # print(file)
#           file_map[file] = os.path.join(path, file)

# def inFolder(elem):
#   if elem in file_map:
#     return True
#   # print(elem)
#   return False

# # remove images which arent there
# for elem in data["images"]:
#   if inFolder(elem["file_name"]):
#     data_final["images"].append(elem)

# for elem in data["annotations"]:
#   if inFolder(elem["image_id"]+".jpg"):
#     data_final["annotations"].append(elem)

# print(len(data["images"]),len(data["annotations"]))
# print(len(data_final["images"]),len(data_final["annotations"]))

# f = open(INSTANCES_TRAIN_ALT_PATH,'w')
# json.dump(data_final,f)


In [0]:
#@title Shift all files to train/

# !ls -l "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/"
# with open(INSTANCES_TRAIN_ALT_PATH) as f:
#   data = json.load(f)
# print(len(data["images"]),len(data["annotations"]))
# !mv "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train.json" "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/temp.json"
# !mv "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_alt.json" "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train.json"

# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images"
# !unzip -q train.zip


# import shutil
# for file in file_map.keys():
#   oldpath = file_map[file]
#   newpath = IMAGE_PATH
#   print(oldpath,newpath)
#   # command = "mv " + oldpath+" "+newpath
#   # os.system(command)
#   shutil.move(oldpath,newpath)
  # !mv {oldpath} {newpath}

/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images
replace train/squirrel/5968bf89-23d2-11e8-a6a3-ec086b02610b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
# !ls -l "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/images/train"
!pwd

In [0]:
#@title Generate annotation CSV

class_map = {
11:"bird",
6:"bobcat",
33:"car",
16:"cat",
3:"raccoon",
10:"rabbit",
9:"coyote",
5:"squirrel"
}

with open(INSTANCES_TRAIN_PATH) as f:
  data = json.load(f)

!rm -rf "{ANNOTATION_CSV}"
f_csv = open(ANNOTATION_CSV, 'w', newline='')
writer = csv.writer(f_csv)

for anno in data["annotations"]:
  bbox = [int(x) for x in anno["bbox"]]
  bbox[2] = bbox[0] + bbox[2]
  bbox[3] = bbox[1] + bbox[3]
  if(anno["category_id"] in class_map):
    class_name = class_map[anno["category_id"]]
    img_path = IMAGE_PATH +"/"+class_name+"/" +anno["image_id"] + ".jpg"
    # print(img_path)
    writer.writerow([img_path,bbox[0],bbox[1],bbox[2],bbox[3],class_name])


In [0]:
!wc -l "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv"

1739 /content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv


In [0]:
#@title Train-Val Split
# # Train Val split
# from numpy.random import RandomState
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv',header=None)
rng = RandomState()

train = df.sample(frac=0.7, random_state=rng)
test = df.loc[~df.index.isin(train.index)]


# !rm -rf "{TRAIN_CSV}"
# !rm -rf "{VAL_CSV}"
train.to_csv(TRAIN_CSV,index=False,header=None)
test.to_csv(VAL_CSV,index=False,header=None)

In [0]:
print("----Whole----")
df = pd.read_csv('/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data/annotations/instances_train_csv.csv',header=None)
print(df[5].value_counts())
print("----Train-----")
df = pd.read_csv(TRAIN_CSV,header=None)
print(df[5].value_counts())
print("----Val----")
df = pd.read_csv(VAL_CSV,header=None)
print(df[5].value_counts())

In [0]:
!cat "{VAL_CSV}"

In [0]:
f_csv = open(CLASS_LIST, 'w', newline='')
writer = csv.writer(f_csv)
lines = [
["bird","0"],
["bobcat","1"],
["car","2"],
["cat","3"],
["raccoon","4"],
["rabbit","5"],
["coyote","6"],
["squirrel","7"]
]
# for line in lines:
#   writer.writerow(line)

In [0]:
# !python visualize.py --dataset csv --csv_classes "{CLASS_LIST}" --csv_val "{ANNOTATION_CSV}" --model "{PATH_TO_WEIGHTS}"
!python train.py --dataset csv --csv_train "{ANNOTATION_CSV}"  --csv_classes "{CLASS_LIST}"

CUDA available: True
No validation annotations provided.
Num training images: 1693
Epoch: 0 | Iteration: 0 | Classification loss: 1.14594 | Regression loss: 1.15697 | Running loss: 2.30291
Epoch: 0 | Iteration: 1 | Classification loss: 1.15146 | Regression loss: 0.97798 | Running loss: 2.21617
Epoch: 0 | Iteration: 2 | Classification loss: 1.15887 | Regression loss: 0.99550 | Running loss: 2.19557
Epoch: 0 | Iteration: 3 | Classification loss: 1.13337 | Regression loss: 1.01302 | Running loss: 2.18327
Epoch: 0 | Iteration: 4 | Classification loss: 1.14780 | Regression loss: 1.03639 | Running loss: 2.18346
Traceback (most recent call last):
  File "train.py", line 173, in <module>
    main()
  File "train.py", line 131, in main
    loss.backward()
  File "/usr/local/lib/python3.6/dist-packages/torch/tensor.py", line 195, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py", line 99,

In [0]:

%matplotlib inline

# !python visualize.py --dataset csv --csv_classes "{CLASS_LIST}"  --csv_val "{VAL_CSV}" --model "{PATH_TO_VWEIGHTS}"

In [0]:
# Helper for visualize


In [7]:
# print(confusion_matrix([[0,0], [1,0], [1,0], [1,1]], [[0,0], [1,0], [0,1], [1,1]]))
# !rm -rf images1/*
# !ls images1/
# !zip -r /content/file.zip ./images1/
# from google.colab import files
# files.download(TO_SAVE_WT)

KeyboardInterrupt: ignored

In [0]:
def draw_caption(image, box, caption):
  b = np.array(box).astype(int)
  cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
  cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

parser = {"csv_classes":CLASS_LIST, "csv_val":VAL_CSV, "model_finetuned":BEST_WEIGHTS, "model_raw": PATH_TO_WEIGHTS}
dataset_val = CSVDataset(train_file=parser["csv_val"], class_list=parser["csv_classes"], transform=transforms.Compose([Normalizer(), Resizer()]))

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

retinanet_raw = model.resnet50(num_classes=8)

retinanet_raw.load_state_dict(torch.load(parser["model_raw"]))
retinanet_finetuned = torch.load(parser["model_finetuned"])
torch.save(retinanet_finetuned.state_dict(), TO_SAVE_WT)


def get_bbox(retinanet,img,bcolor,data,caption):
  scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
  # print("--------Classification-----------")
  # print(classification.shape,classification)
  # print("--------Scores----------")
  # print(scores.shape,scores)
  # print("---------Transformed_Idxs-------")
  # print(transformed_anchors.shape, transformed_anchors)
  idxs = np.where(scores.cpu()>0.5)
  # print(idxs[0])
  # bbox = transformed_anchors[idxs[0][j], :]
  class_labels = []
  for j in range(idxs[0].shape[0]):
      bbox = transformed_anchors[idxs[0][j], :]
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(classification[idxs[0][j]])]
      draw_caption(img, (x1, y1, x2, y2), label_name)
      class_labels.append(label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=bcolor, thickness=2)
      # print(caption,label_name)
  return class_labels


def equalize(pred, truth):
  diff = len(pred) - len(truth)
  nothing_pad = ["nothing" for x in range(abs(diff))]
  if(diff > 0):
    truth += nothing_pad
  elif(diff < 0):
    pred += nothing_pad
  return pred, truth
  
use_gpu = True

if use_gpu:
  retinanet_raw = retinanet_raw.cuda()
  retinanet_finetuned = retinanet_finetuned.cuda()

retinanet_raw.eval()
retinanet_finetuned.eval()

unnormalize = UnNormalizer()

# fig, ax = plt.subplots(figsize=(20, 10))
classcnt = [3 for x in range(8)]
sum = 24


raw_pred_list = []
finetune_pred_list = []
gt_raw_list = []
gt_finetune_list = []
cnt = 1 
for idx, data in enumerate(dataloader_val):
  # print(int(data['annot'][0][0][4]))

  # 3 images for each class
  # category = int(data['annot'][0][0][4])
  # if(classcnt[category] > 0):
  #   classcnt[category] -= 1
  #   sum -= 1
  # elif sum <= 0:
  #   break
  # else:
  #   continue
  
  with torch.no_grad():
    
    img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

    img[img<0] = 0
    img[img>255] = 255

    img = np.transpose(img, (1, 2, 0))

    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

    # st = time.time()
    raw_pred = get_bbox(retinanet_raw,img,(0,0,255),data,str(cnt)+" Model:")
    finetune_pred = get_bbox(retinanet_finetuned,img,(0,255,0),data,str(cnt)+" Finetuned model:")
    gt_raw = []
    gt_finetune = []
    # Ground truth
    # print(data['annot'])

    for bbox in data['annot'][0]:
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      label_name = dataset_val.labels[int(bbox[4])]
      gt_raw.append(label_name)
      gt_finetune.append(label_name)
      draw_caption(img, (x1, y1, x2, y2), label_name)
      # print(str(cnt)+" Ground truth:",label_name)
      cv2.rectangle(img, (x1, y1), (x2, y2), color=(255,0,0), thickness=2)
      # print(label_name)

    raw_pred,gt_raw = equalize(raw_pred,gt_raw)
    finetune_pred,gt_finetune = equalize(finetune_pred,gt_finetune)

    # print("Raw pretrained",raw_pred,"Ground",gt_raw)
    # print("Finetune pred",finetune_pred,"Ground",gt_finetune)

    raw_pred_list += raw_pred
    finetune_pred_list += finetune_pred

    gt_raw_list += gt_raw
    gt_finetune_list += gt_finetune

    # print("Ground truth",gt)
    # print("------------------------")
    # print('Elapsed time: {}'.format(time.time()-st))

    # plt.figure(figsize=(20,10)) 
    # plt.imshow(img)
    # plt.savefig("images1/"+str(cnt)+"classification.png")
    # print("---------------------------------")
    cnt += 1
    # plt.show()

# print(len(raw_pred_list),len(gt_raw_list))
# print(len(finetune_pred_list),len(gt_finetune_list))

# print((raw_pred_list),(gt_raw_list))
# print((finetune_pred_list),(gt_finetune_list))

print(confusion_matrix(gt_raw_list,raw_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))
print(confusion_matrix(gt_finetune_list,finetune_pred_list,labels=["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel","nothing"]))

In [0]:
%%time



parser = {"csv_classes":CLASS_LIST, "csv_val":VAL_CSV, "csv_train":TRAIN_CSV, "epochs":15 }

dataset_train = CSVDataset(train_file=parser["csv_train"], class_list=parser["csv_classes"],
                                   transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
dataset_val = CSVDataset(train_file=parser["csv_val"], class_list=parser["csv_classes"],
                                     transform=transforms.Compose([Normalizer(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_train, batch_size=2, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)

retinanet = model.resnet50(num_classes=dataset_train.num_classes(),)

for param in retinanet.parameters():
    param.requires_grad = False
for param in retinanet.classificationModel.output.parameters():
    param.requires_grad = True
for param in retinanet.regressionModel.output.parameters():
    param.requires_grad = True

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

PATH_TO_DATASET = "/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/assignment-data"
PATH_TO_WEIGHTS = PATH_TO_DATASET + "/pretrained_weights_cleaned.pt"

retinanet= torch.load(BEST_WEIGHTS)

retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, retinanet.parameters()), lr=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

loss_hist = collections.deque(maxlen=500)

loss_all_epochs = []

retinanet.train()
retinanet.module.freeze_bn()
file1 = open("logfreezenew.txt","w")
print('Num training images: {}'.format(len(dataset_train)))

mAP = csv_eval.evaluate(dataset_val, retinanet)


# best_loss = 100
# for epoch_num in range(parser["epochs"]):
#     print("\nEpoch",epoch_num)
#     retinanet.train()
#     retinanet.module.freeze_bn()

#     if epoch_num > 10:
#       for param in retinanet.parameters():
#         param.requires_grad = True
#       optimizer = optim.Adam(filter(lambda p: p.requires_grad, retinanet.parameters()), lr=1e-5)
    
#     epoch_loss = []

#     for iter_num, data in enumerate(dataloader_train):
#         try:
#             optimizer.zero_grad()

#             classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])

#             classification_loss = classification_loss.mean()
#             regression_loss = regression_loss.mean()

#             loss = classification_loss + regression_loss

#             if bool(loss == 0):
#                 continue

#             loss.backward()

#             torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)

#             optimizer.step()

#             loss_hist.append(float(loss))

#             epoch_loss.append(float(loss))

#             # print(
#             #     'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
#             #         epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
            
#             file1.write('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f} \n'.format(
#                     epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
            
#             del classification_loss
#             del regression_loss
#         except Exception as e:
#             print(e)
#             continue

#     print('Evaluating dataset')

#     mAP = csv_eval.evaluate(dataset_val, retinanet)

#     mean_loss = np.mean(epoch_loss)

#     scheduler.step(mean_loss)

#     loss_all_epochs.append(mean_loss)

#     if(mean_loss < best_loss):
#         torch.save(retinanet.module, 'freeze_model/{}_retinanet_{}.pt'.format("csv", "best_model"))
#         best_loss = mean_loss
    
#     torch.save(retinanet.module, 'freeze_model/{}_retinanet_{}.pt'.format("csv", epoch_num))

# retinanet.eval()
# file1.close()
# torch.save(retinanet, 'freeze_model/model_final.pt')

# print("Loss curve")
# x_axis = [x for x in range(15)]
# plt.plot(x_axis,loss_all_epochs)
# plt.xlabel("Epochs")
# plt.ylabel("Running Loss")
# plt.savefig("/content/drive/My Drive/Deep_Learning_Assignments/Assignment1/Q3/dlassignment1/q3/freeze_model/losscurve.png")

Num training images: 1153

mAP:
bird: 0.3391735656587355
bobcat: 0.7082753665581452
car: 0.9995748299319728
cat: 0.6007539827457286
raccoon: 0.8516176272969179
rabbit: 0.5223462303178512
coyote: 0.5125965089552045
squirrel: 0.5644058690910918
0.637342997569456
CPU times: user 5min 3s, sys: 1min 5s, total: 6min 9s
Wall time: 5min 12s
